Follow [here](https://github.com/pangolulu/rnn-from-scratch) - RNN and [here](http://www.wildml.com/2015/10/recurrent-neural-networks-tutorial-part-3-backpropagation-through-time-and-vanishing-gradients/)

Follow [here](http://nicodjimenez.github.io/2014/08/08/lstm.html) - LSTM and [here](https://github.com/nicodjimenez/lstm). And mostly [here](http://blog.varunajayasiri.com/numpy_lstm.html).

## Sending forward

At each step in an RNN, the cells take in a state $S$, a weight matrix $W$ multiplied by $S$, an input $X$, an weight matrix $U$. The RNN layer has a certain "breadth", that determines both the second dimensions of these weight matrices. 

The input $X$ will have the dimensions of the number of words, num_words, and $U$ will be used to transform this into the hidden layer, so $U$ will have dimension (num_words, num_hidden).

The state for a given layer will be a vector of its number of hidden units.

Each layer will have a state from the prior time step. This state can just be initialized to be all zeros.

This state is then multiplied by the weight matrix $W$. This weight matrix has dimension (hidden_dim, hidden_dim). 

These two vectors, $WS$ and $XU$ are then added. Then, they are multiplied by a third weight matrix $V$ to get the output state for the layer.

This state $SV$ is then used in the next layer as the layer's output $O$.

So - this is key - at each time step, each layer in the RNN returns a state $S$ and an output $O$ that is actually passed onto the next layer.

In addition, it produces $V$, which has dimension (num_hidden, num_words), that produces an output softmax $O$ over the number of words. 

When figuring out how backpropogation works, we can assume that we get from the layer above how this state $S$ and this output $O$ are affecting the loss; from this, we'll need to figure out how to update the weights $V$, $U$, and $W$.

Once backpropagation is complete, each layer sends down to the layer below it a gradient for how much the prior state affects the output.

In [2]:
import numpy as np

class MultiplyGate:
    def forward(self, W, X):
        return np.dot(X, W)

    def backward(self, W, X, dZ):
        dW = np.dot(np.transpose(X), dZ)
        dX = np.dot(dZ, np.transpose(W))
        return dW, dX

class AddGate:
    def forward(self, X, b):
        return X + b

    def backward(self, X, b, dZ):
        dX = dZ * np.ones_like(X)
        db = np.dot(np.ones((1, dZ.shape[0]), dtype=np.float64), dZ)
        return db, dX

In [1]:
class RNNLayer:
    def forward(self, x, prev_s, U, W, V):
        # x = 10000 x 1
        # prev_s = 100 x 1
        # U = 10000 x 100
        # W = 100 x 100
        # V = 100 x 100
        self.mulu = mulGate.forward(U, x)
        self.mulw = mulGate.forward(W, prev_s)
        self.add = addGate.forward(self.mulw, self.mulu)
        self.s = activation.forward(self.add)
        self.mulv = mulGate.forward(V, self.s)
        
    def backward(self, x, prev_s, U, W, V, diff_s, dmulv):
        # Each forward pass receives 4 things: 
        # previous state
        # U
        # V
        # W
        # In principle, each of these quantities affects the output *of this layer* in some way
        # And each of these should then be updated by a function of how they affect the output
        # of this layer and of the gradient this layer receives 
        self.forward(x, prev_s, U, W, V)
        dV, dsv = mulGate.backward(V, self.s, dmulv)
        # V = 100 x 100
        # sv = 100
        # dsv = np.dot(dmulv, np.transpose(V))
        # diff_s -> received from the layer above. Both of these are the same shape as S
        ds = dsv + diff_s
        # dadd ->  sending back through activation
        dadd = activation.backward(self.add, ds)
        # sending backward again...
        dmulw, dmulu = addGate.backward(self.mulw, self.mulu, dadd)
        # dW, 
        dW, dprev_s = mulGate.backward(W, prev_s, dmulw)
        dU, dx = mulGate.backward(U, x, dmulu)
        return (dprev_s, dU, dW, dV)

# LSTMs

In LSTMs, there's a "concatenation" step: the $X$ values, with dimensions of the number of words in the vocabulary, and the $S$ values, with dimensions the number of hidden values in the state, are concatenated together before being multiplied by a matrix. Note that this is equivalent to the two values being each multiplied by their own matrices and then the results being concatenated together: this does not "weight" the words more than the state, for example.

## Similarity with regular RNNs

LSTMs look the same as regular RNNs from the outside: each cell outputs the results of its operations forward both to the next layer, and back to the current layer; this output (often denoted $h_t$) has dimension equal to the dimension of the LSTM layer.

The difference is in how the weights are updated inside the layer. LSTM layers maintain a "cell state" that stores information in a persistent manner. As they receive both real world data and the output from the prior layer, they perform a series of operations that ultimately result in outputting a vector of hidden states, but do so by operating on the cell state. Let's take a look:

## LSTM, step by step

Let's say we have a vocabulary of 10,000 words, and an LSTM layer with 100 nodes. What actually goes on in the forward pass?

First: the input vector of length 10,000 is passed in, along with the cell hidden state of length 100.

$X_t$ and $h_{t-1}$

Then, these are concatenated into a vector of length 10,100, and multiplied together and fed through a sigmoid to create a forget gate $f_t$.

$f_t = \sigma(W_f * [X_t, h_{t-1}])$

Then, similarily, the input and new candidate cell states are created:

$i_t = \sigma(W_i * [X_t, h_{t-1}])$

$C^{new}_t = tanh(W_C * [X_t, h_{t-1}])$

The cell state is updated:

$ C_t = f_t * C_{t-1} + i_t * C^{new}_t $

Finally, another output is created:

$o_t = \sigma(W_o * [X_t, h_{t-1}])$

And this is used to update the hidden state:

$h_t = o_t * tanh(C_t)$

Note: all of these W matrices are 10,100 x 100. $C_t$ and all the other vectors are of dimension 100.

Finally, if we are outputing a softmax layer over the words, we must output a layer of length 10,000. We'll call this $V_t$. If

$ V_t = W_V * h_t $ 

Then $W_V$ has dimension 100 x 10,000

## Backpropagation

The question now is: how does backpropagation work?

To start: in backpropagation for LSTMs, we'll receive:

* The gradient for the cell state $C$. 
* The gradient for the output $V$.
* The gradient for the hidden state $h$.

Using these, we can trace backwards through the LSTM and get the gradients for everything else.

We can update all the gradients in the following order:

1. $V_{grad}$ (from $y$) 
1. $Wv_{grad}$ (from $V_{grad}$ and $h_t$)
1. $h_{grad}$ (from $W_grad$ and $h_{grad}$)
1. $o_{grad}$ (from $h_{grad}$ and $C_t$)
1. $Wo_{grad}$ (from $o_{grad}$ and $Z_t$)
1. $C_{grad}$ (from $h_{grad}$ and $C_t$ (input), and $o_t$)
1. $C^{new}_{grad}$ (from $i$ and $C_{grad}$ and $C^{new}_t$)
1. $Wc_{grad}$ (from $C^{new}_{grad}$ and $Z_t$)
1. $i_{grad}$ (from $C_{grad}$ and $C^{new}_t$)
1. $Wi_{grad}$ (from $i_{grad}$ and $Z_t$)
1. $f_{grad}$ (from $C_{grad}$ and $C^{new}_t$)
1. $Wf_{grad}$ (from $f_{grad}$ and $Z_t$)
1. $Z_{grad}$ (from the four weight matrices, and $f_t$, $i_t$, $C^{new}_t$, $o_t$)
1. $C^{in}_{grad}$ (from $f_t$ and $C_{grad}$)

Goal: get to $C^{in}_{grad}$ and $h^{in}_{grad}$. Remember that $h^{in}$ is just a part of $Z$, which is multiplied by several inputs throughout the internals of the LSTM cell. So, if we get $Z_{grad}$, then we can extract $h_{grad}$ and $x_{grad}$.

In [ ]:
def backward(target, dh_next, dC_next, C_prev,
             z, f, i, C_bar, C, o, h, v, y,
             p = parameters):
    
    assert z.shape == (X_size + H_size, 1)
    assert v.shape == (X_size, 1)
    assert y.shape == (X_size, 1)
    
    for param in [dh_next, dC_next, C_prev, f, i, C_bar, C, o, h]:
        assert param.shape == (H_size, 1)

    # 1
    dv = np.copy(y)
    dv[target] -= 1

    # 2
    p.W_v.d += np.dot(dv, h.T)
    p.b_v.d += dv

    # 3
    dh = np.dot(p.W_v.v.T, dv)        
    dh += dh_next

    # 4
    do = dh * tanh(C)
    do = dsigmoid(o) * do
    
    # 5
    p.W_o.d += np.dot(do, z.T)
    p.b_o.d += do

    # 6
    dC = np.copy(dC_next)
    dC += dh * o * dtanh(tanh(C))

    # 7
    dC_bar = dC * i
    dC_bar = dtanh(C_bar) * dC_bar

    # 8
    p.W_C.d += np.dot(dC_bar, z.T)
    p.b_C.d += dC_bar

    # 9
    di = dC * C_bar
    di = dsigmoid(i) * di
    
    # 10
    p.W_i.d += np.dot(di, z.T)
    p.b_i.d += di

    # 11    
    df = dC * C_prev
    df = dsigmoid(f) * df

    # 12
    p.W_f.d += np.dot(df, z.T)
    p.b_f.d += df

    # 13
    dz = (np.dot(p.W_f.v.T, df)
         + np.dot(p.W_i.v.T, di)
         + np.dot(p.W_C.v.T, dC_bar)
         + np.dot(p.W_o.v.T, do))
    
    # 14
    dh_prev = dz[:H_size, :]
    dx_prev = dz[H_size:, :]
    dC_prev = f * dC
    
    return dh_prev, dC_prev, dx_prev

## BPTT

The backpropogation through time algorithm works the same way for RNNs as for LSTMs. 

The difference is that regular RNN cells compute and send backwards a hidden state over time.

LSTM cells compute (and send backwards) both a hidden state and a cell state. 

In both of these, to do backpropagation properly, we must keep track of the hidden states and cell states over time. The length that we keep track of is equal to the length of the input sequences we feed in.

## Deep Dive into Forward Pass

Goal: for a given cell, a forward pass should take in

* $X_t$ (of dimension e.g. 10,000) 
* $h_{t-1}$ (of dimension 100)
* $C_{t-1}$ (of dimension 100)

For the layer, it should output:

* $y_t$, the prediction for that layer at time $t$ (of dimension 10,000).
* $h_t$, the output for that layer at time $t$ (of dimension 100).
* $C_t$, the cell state for that layer at time $t$ (of dimension 10,100).

### Step 1:

Concatenate $X_t$ and $h_{t-1}$.

`z = np.row_stack((h_prev, x))`

### Step 2:

Compute $f_t = \sigma(W_f * [X_t, h_{t-1}])$.

`f = sigmoid(np.dot(p.W_f.v, z) + p.b_f)`

Note: $W_f$ has dimension 10,100 x 100.

### Step 3:

Compute $i_t = \sigma(W_i * [X_t, h_{t-1}])$.

`i = sigmoid(np.dot(p.W_i.v, z) + p.b_i.v)`

Note: $W_i$ has dimension 10,100 x 100.

### Step 4:

Compute $\tilde{C_t} = tanh(W_c * [X_t, h_{t-1}])$

`C_tilde = tanh(np.dot(p.W_C.v, z) + p.b_C.v)`

Note: $W_c$ has dimension 10,100 x 100.

### Step 5:

Compute $ C_t = f_t * C_{t-1} + i_t * \tilde{C_t} $

`C = f * C_prev + i * C_tilde`

### Step 6:

Compute $o_t = \sigma(W_o * [X_t, h_{t-1}])$

`o = sigmoid(np.dot(p.W_o.v, z) + p.b_o.v)`

Note: $W_o$ has dimension 10,100 x 100.

### Step 7:

Compute $h_t = o_t * tanh(C_t)$

`h = o * tanh(C)`

### Step 8:

Compute $ V_t = W_V * h_t $ 

`v = np.dot(p.W_v.v, h) + p.b_v.v`

Note: $W_v$ has dimension 100 x 10,000.

### Step 9:

Compute $ Y_t = softmax(V_t) $ 

In [ ]:
def forward(x, h_prev, C_prev, p = parameters):
    
    # Step 1
    z = np.row_stack((h_prev, x))
    
    # Step 2
    f = sigmoid(np.dot(p.W_f.v, z) + p.b_f.v)
    
    # Step 3
    i = sigmoid(np.dot(p.W_i.v, z) + p.b_i.v)
    
    # Step 4
    C_tilde = tanh(np.dot(p.W_C.v, z) + p.b_C.v)

    # Step 5 
    C = f * C_prev + i * C_tilde

    # Step 6
    o = sigmoid(np.dot(p.W_o.v, z) + p.b_o.v)

    # Step 7
    h = o * tanh(C)

    # Step 8
    v = np.dot(p.W_v.v, h) + p.b_v.v

    # Step 9
    y = np.exp(v) / np.sum(np.exp(v)) #softmax

    return z, f, i, C_bar, C, o, h, v, y

## Deep Dive into Backward Pass

Each LSTM layer on the backward pass, at each time step, will take in:

The gradients for

* $y_t$ (of dimension e.g. 10,000) 
* $h_t$ (of dimension 100)
* $C_t$ (of dimension 100)

That is, the partial derivatives of these with respect to the loss.

$$\frac{\partial{L}}{\partial{Y}}$$

$$\frac{\partial{L}}{\partial{H}}$$

$$\frac{\partial{L}}{\partial{C}}$$

Note: for the last character in the sequence, these last two have to be initialized to 0.

For the layer, it should output the gradients for

* $x_t$, the prediction for that layer at time $t$ (of dimension 10,000).
* $h_{t_1}$, the output for that layer at time $t$ (of dimension 100).
* $C_{t-1}$, the cell state for that layer at time $t$ (of dimension 10,100).

## Step 1

`target` is a one hot encoded vector of output.
`y` is the predicted output.

The loss is a function of `y-target`. 

$ y = softmax(v) $.

A softmax layer has no gradient, so:

$$\frac{\partial{L}}{\partial{V}} = -(y -  y^{hat})$$

## Step 2

Recall in our basic neural net, when we did $ W * B = C $, and we were calculating $ \frac{\partial{L}}{\partial{W}} $ and $ \frac{\partial{L}}{\partial{B}} $, using $ \frac{\partial{L}}{\partial{C}} $, it turned out that:

$$ \frac{\partial{C}}{\partial{W}} = B^T $$

and

$$ \frac{\partial{C}}{\partial{B}} = W^T $$

So that, for example, because of the chain rule,

$$ \frac{\partial{L}}{\partial{W}} = \frac{\partial{L}}{\partial{C}} * \frac{\partial{C}}{\partial{W}} = \frac{\partial{L}}{\partial{W}} = \frac{\partial{L}}{\partial{C}} * B^T $$



So, since $ V_t = W_v * h_t $, 

We have both:

$$ \frac{\partial{L}}{\partial{W}} = \frac{\partial{L}}{\partial{V}} * H^T $$

and

$$ \frac{\partial{L}}{\partial{H}} = \frac{\partial{L}}{\partial{V}} * W_v^T $$

## Step 3

To the current value of $ \frac{\partial{L}}{\partial{H}} $, we add the gradient from the next time step.

$$ \frac{\partial{L}}{\partial{H}} = \frac{\partial{L}}{\partial{H}} + H_{grad} $$

## Step 4

Next we want to compute

$$ \frac{\partial{L}}{\partial{O}} $$

We have both $h_t = o_t * tanh(C_t)$

and by the logic presented for step 2, we have:

$$ \frac{\partial{L}}{\partial{O}} = \frac{\partial{L}}{\partial{H}} * tanh(C_t) $$

## Step 4.5 

Call $ W_o * [X_t, h_{t-1}] = p$. To continue "moving down the chain", we want to compute

$$ \frac{\partial{L}}{\partial{P}} $$ 

We know that since $ o_t = \sigma(p) $, so the derivative of this function is $ \sigma(p) * (1 - \sigma(p)) $. Thus

$$ \frac{\partial{L}}{\partial{P}} = \frac{\partial{L}}{\partial{O}} * \sigma(p) * (1 - \sigma(p)) $$

## Step 5

Similarly to step 2, we have

$ W_o * [X_t, h_{t-1}] = p $

We'll see this quantity $[X_t, h_{t-1}]$ quite a bit, so we'll call it $z_t$.

Since 

$ W_o * z_t = p $

We have both:

$$ \frac{\partial{L}}{\partial{z_t}} = \frac{\partial{L}}{\partial{P}} * W_o^T $$

(though as we shall see, this is just one component of $\frac{\partial{L}}{\partial{z_t}}$) - and also:

$$ \frac{\partial{L}}{\partial{W_o}} = \frac{\partial{L}}{\partial{P}} * z^T $$

## Step 6:

Next we want to compute $$ \frac{\partial{L}}{\partial{C_t}} $$, and we have 

$h_t = o_t * tanh(C_t)$

First, we receive $C_{grad}$ from the layer above, so we can start by initializing

$$ \frac{\partial{L}}{\partial{C_t}} = C_{grad} $$

First, set $ tanh(C_t) = D_t $. Then, using exactly the same logic as in Step 2:

$$ \frac{\partial{L}}{\partial{D_t}} = \frac{\partial{L}}{\partial{H_t}} * o_t $$.

Then, since $ D_t = tanh(C_t) $, the derivative of this will be the derivative of the `tanh` function evaluated at $tanh(C_t)$. So, applying the chain rule:

$$ \frac{\partial{L}}{\partial{C_t}} = \frac{\partial{L}}{\partial{D_t}} * \frac{\partial{D_t}}{\partial{C_t}} = \frac{\partial{L}}{\partial{H_t}} * o_t * tanh'(tanh(C_t)) $$

## Step 7

Next we want to compute $$ \frac{\partial{L}}{\partial{\tilde{C_t}}} $$

Since $C_t = \tilde{C_t} * i_t$, where this represents an element-wise multiplication, we have simply:

$ \frac{\partial{L}}{\partial{\tilde{C_t}}} = \frac{\partial{L}}{\partial{C}} * i_t $ 

## Step 7.5

Next we want to ultimately compute $$ \frac{\partial{L}}{\partial{W_c}} $$. 

Following Step 4, we'll first define an intermediate quantity $ E_t = W_c * [X_t, h_{t-1}]$, and compute $ \frac{\partial{L}}{\partial{E_t}} $.

We have $ \tilde{C_t} = tanh(E_t)$, so 

$$ \frac{\partial{L}}{\partial{E_t}} = \frac{\partial{L}}{\partial{C_t}} * tanh'(tanh(E_t)) $$

## Step 8

Next, we'll compute $$ \frac{\partial{L}}{\partial{W_c}} $$. Similarly to Step 5, since:

$$ E_t = W_c * z_t $$

We have both:

$$ \frac{\partial{L}}{\partial{W_c}} = \frac{\partial{L}}{\partial{E_t}} * z^T $$

(multiplying by the transpose of $z$ since these are again matrix multiplications), and 

$$ \frac{\partial{L}}{\partial{Z}} = \frac{\partial{L}}{\partial{E_t}} * W_c^T $$

## Step 9

Step 9 follows Step 7. We want to compute $$ \frac{\partial{L}}{\partial{\tilde{i_t}}} $$

Since $C_t = \tilde{C_t} * i_t$, where this represents an element-wise multiplication, we have simply:

$ \frac{\partial{L}}{\partial{\tilde{i_t}}} = \frac{\partial{L}}{\partial{C}} * \tilde{C_t} $.

## Step 9.5

Similarly, step 9.5 follows Step 7.5. We ultimately want to compute $$ \frac{\partial{L}}{\partial{W_i}} $$. 

Following Step 4, we'll first define an intermediate quantity $ J_t = W_i * [X_t, h_{t-1}]$, and compute $ \frac{\partial{L}}{\partial{J_t}} $.

We have $i_t  = \sigma(J_t)$, so 

$$ \frac{\partial{L}}{\partial{J_t}} = \frac{\partial{L}}{\partial{i_t}} * \sigma(J_t) * (1-\sigma(J_t)) $$

## Step 10

Step 10 is identical to Step 8: we can now compute $$ \frac{\partial{L}}{\partial{W_i}} $$. 
Similarly to Step 5, since:

$$ J_t = W_i * z_t $$

We have both:

$$ \frac{\partial{L}}{\partial{W_i}} = \frac{\partial{L}}{\partial{J_t}} * z^T $$

and, adding to rolling sum of $ \frac{\partial{L}}{\partial{Z}} $, 

$$ \frac{\partial{L}}{\partial{Z_t}} = \frac{\partial{L}}{\partial{J_t}} * W_i^t $$

## Step 11

Step 11 follows Step 7. We want to compute $$ \frac{\partial{L}}{\partial{\tilde{f_t}}} $$

Since $ C_t = \tilde{f_t} * C_{prev}$, where this represents an element-wise multiplication, we have simply:

$$ \frac{\partial{L}}{\partial{f_t}} = \frac{\partial{L}}{\partial{C_t}} * C_{prev} $$

## Step 11.5

Similarly, step 11.5 follows step 7.5. We want to ultimately compute $$ \frac{\partial{L}}{\partial{W_f}} $$. 

Following Step 4, we'll first define an intermediate quantity $ G_t = W_f * [X_t, h_{t-1}]$, and compute $ \frac{\partial{L}}{\partial{G_t}} $.

We have $ f_t = \sigma(G_t)$, so 

$$ \frac{\partial{L}}{\partial{G_t}} = \frac{\partial{L}}{\partial{f_t}} * \sigma(G_t) * (1-\sigma(G_t)) $$

## Step 12

Step 12 is identical to Steps 8 and 10: we can now compute $$ \frac{\partial{L}}{\partial{W_i}} $$. 

Similarly to Step 5, since:

$$ G_t = W_f * z_t $$

We have both:

$$ \frac{\partial{L}}{\partial{W_f}} = \frac{\partial{L}}{\partial{G_t}} * z^T $$

and, adding to rolling sum of $ \frac{\partial{L}}{\partial{Z_t}} $, 

$$ \frac{\partial{L}}{\partial{Z_t}} = \frac{\partial{L}}{\partial{G_t}} * W_f^t $$

## Step 13

Step 13 follows steps 11 and 7. We want to compute $$ \frac{\partial{L}}{\partial{C_{prev}}} $$

Since $ C_t = f_t * C_{prev}$, where this represents an element-wise multiplication, we have simply:

$$ \frac{\partial{L}}{\partial{C_{prev}}} = \frac{\partial{L}}{\partial{C_t}} * f_t $$

## Step 14

Luckily, this isn't really a step, since we've alread computed the four components of 

$$ \frac{\partial{L}}{\partial{Z_t}} = \frac{\partial{L}}{\partial{G_t}} * W_f^t + \frac{\partial{L}}{\partial{J_t}} * W_i^t + \frac{\partial{L}}{\partial{E_t}} * W_c^T + \frac{\partial{L}}{\partial{P}} * W_o^T $$

## Step 15

On the forward pass, $Z_t$ was simply made up of $X_t$ and $h_{t-1}$ concatenated. So, we can select the appropriate elements of $ \frac{\partial{L}}{\partial{Z_t}} $ to make up $ \frac{\partial{L}}{\partial{X_t}} $ and $ \frac{\partial{L}}{\partial{h_{t-1}}} $.

In [7]:
    def backward(self, Y_grad, H_grad, C_grad, LSTM_params):
        # Initialize the gradient for the words and the hidden layers:
        self.Z_diff = np.zeros_like(self.Z)
        
        # 2
        LSTM_params.W_V_diff += np.dot(Y_grad, self.H.T)

        # 3
        self.H_diff = np.dot(LSTM_params.W_V_diff.T, Y_grad)
        self.H_diff += H_grad

        # 4
        self.O_diff = self.H_diff * self.C_act

        # 4.5
        self.O_int_diff = sigmoid(self.O, deriv=True) * self.O
        
        # 5
        LSTM_params.W_O_diff += np.dot(self.O_int_diff, self.Z.T)
        self.Z_diff = np.dot(self.O_int_diff, LSTM_params.W_O.T)
        
        # 6
        self.C_diff = C_grad
        self.C_diff += self.H_diff * self.O * tanh(C_act, deriv=True)

        # 7
        self.C_prop_diff = self.C_diff * self.I

        # 7.5
        self.C_prop_int_diff = tanh(self.C_prop, deriv = True) * self.C_prop_diff

        # 8
        LSTM_params.W_C_diff += np.dot(self.C_prop_int_diff, self.Z.T)
        self.Z_diff += np.dot(self.C_prop_int, LSTM_params.W_C.T)
        
        # 9
        self.I_diff = self.C_diff * self.C_prop

        # 9.5
        self.I_int_diff = sigmoid(self.I, deriv=True) * self.I_diff

        # 10
        LSTM_params.W_I_diff += np.dot(self.I_int_diff, self.Z.T)
        self.Z_diff += np.dot(self.I_int, LSTM_params.W_I.T)
        
        # 11
        self.F_diff = self.C_diff * self.C

        # 11.5
        self.F_int_diff = sigmoid(self.F, deriv=True) * self.F_diff

        # 12
        LSTM_params.W_F_diff += np.dot(self.F_int, self.Z.T)        
        self.Z_diff += np.dot(self.F_int, LSTM_params.W_F.T)
        
        # 13
        C_grad = self.F * self.C_diff

        # 14
        X_grad = self.Z[:self.vocab_size, :]
        H_grad = self.Z[self.vocab_size:, :]        

        return H_grad, C_grad, X_grad

In [ ]:
def backward(target, dh_next, dC_next, C_prev,
             z, f, i, C_bar, C, o, h, v, y,
             p = parameters):
    
    assert z.shape == (X_size + H_size, 1)
    assert v.shape == (X_size, 1)
    assert y.shape == (X_size, 1)
    
    for param in [dh_next, dC_next, C_prev, f, i, C_bar, C, o, h]:
        assert param.shape == (H_size, 1)

    # 1
    dv = np.copy(y)
    dv[target] -= 1

    # 2
    p.W_v.d += np.dot(dv, h.T)
    p.b_v.d += dv

    # 3
    dh = np.dot(p.W_v.v.T, dv)        
    dh += dh_next

    # 4
    do = dh * tanh(C)
    
    # 4.5
    dPt = dsigmoid(o) * do
    
    # 5
    p.W_o.d += np.dot(dPt, z.T)
    p.b_o.d += dPt

    # 6
    dC = np.copy(dC_next)
    dC += dh * o * dtanh(tanh(C))

    # 7
    dC_bar = dC * i
    
    # 7.5
    dEt = dtanh(C_bar) * dC_bar

    # 8
    p.W_C.d += np.dot(dEt, z.T)
    p.b_C.d += dEt

    # 9
    di = dC * C_bar
    
    # 9.5
    dJt = dsigmoid(i) * di
    
    # 10
    p.W_i.d += np.dot(dJt, z.T)
    p.b_i.d += di

    # 11
    df = dC * C_prev
    
    # 11.5
    dGt = dsigmoid(f) * df

    # 12
    p.W_f.d += np.dot(dGt, z.T)
    p.b_f.d += df

    # 13
    dC_prev = f * dC
    
    # 14
    dz = (np.dot(p.W_f.v.T, dGt)
         + np.dot(p.W_i.v.T, dJt)
         + np.dot(p.W_C.v.T, dEt)
         + np.dot(p.W_o.v.T, d))
    
    # 15
    X_ = dz[:self.vocab_size, :]
    dx_prev = dz[:self.vocab_size, :]
    
    return dh_prev, dC_prev, dx_prev

# Testing

We want a series of text. We'll break it into 15 character sequences, and learn to predict the next character in the sequence.

# What's next

I want to do CNNs and RNNs from scratch - I have the code done for both.

End goal:

## CNNs:

* Explanation of forward propagation, backpropagation with low level code, visuals, and math where necessary.
* Need to fit this in with a whole neural net architecture. 

## RNNs:

* Add visuals to explanation of forward, backward propagation.
* Hardest part (part I'm avoiding): fitting in this RNN explanation into a neural net framework.


## RNN net framework

Can modify this to be about LSTMs:

In [ ]:
mulGate = MultiplyGate()
addGate = AddGate()
activation = Tanh()

class RNNLayer:
    def forward(self, x, prev_s, U, W, V):
        self.mulu = mulGate.forward(U, x)
        self.mulw = mulGate.forward(W, prev_s)
        self.add = addGate.forward(self.mulw, self.mulu)
        self.s = activation.forward(self.add)
        self.mulv = mulGate.forward(V, self.s)
        
    def backward(self, x, prev_s, U, W, V, diff_s, dmulv):
        self.forward(x, prev_s, U, W, V)
        dV, dsv = mulGate.backward(V, self.s, dmulv)
        ds = dsv + diff_s
        dadd = activation.backward(self.add, ds)
        dmulw, dmulu = addGate.backward(self.mulw, self.mulu, dadd)
        dW, dprev_s = mulGate.backward(W, prev_s, dmulw)
        dU, dx = mulGate.backward(U, x, dmulu)
        return (dprev_s, dU, dW, dV)

So we would have `class LSTMLayer`.

Need a function that runs one training example (or batch of examples) through the network.

Something like this:

In [ ]:
def bptt(self, x, y):
    assert len(x) == len(y)
    output = Softmax()
    layers = self.forward_propagation(x)
    dU = np.zeros(self.U.shape)
    dV = np.zeros(self.V.shape)
    dW = np.zeros(self.W.shape)

    T = len(layers)
    prev_s_t = np.zeros(self.hidden_dim)
    diff_s = np.zeros(self.hidden_dim)
    for t in range(0, T):
        dmulv = output.diff(layers[t].mulv, y[t])
        input = np.zeros(self.word_dim)
        input[x[t]] = 1
        dprev_s, dU_t, dW_t, dV_t = layers[t].backward(input, prev_s_t, self.U, self.W, self.V, diff_s, dmulv)
        prev_s_t = layers[t].s
        dmulv = np.zeros(self.word_dim)
        for i in range(t-1, max(-1, t-self.bptt_truncate-1), -1):
            input = np.zeros(self.word_dim)
            input[x[i]] = 1
            prev_s_i = np.zeros(self.hidden_dim) if i == 0 else layers[i-1].s
            dprev_s, dU_i, dW_i, dV_i = layers[i].backward(input, prev_s_i, self.U, self.W, self.V, dprev_s, dmulv)
            dU_t += dU_i
            dW_t += dW_i
        dV += dV_t
        dU += dU_t
        dW += dW_t
    return (dU, dW, dV)

In [1]:
def sgd_step(self, x, y, learning_rate):
    dU, dW, dV = self.bptt(x, y)
    self.U -= learning_rate * dU
    self.V -= learning_rate * dV
    self.W -= learning_rate * dW
   
def train(self, X, Y, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    num_examples_seen = 0
    losses = []
    for epoch in range(nepoch):
        if (epoch % evaluate_loss_after == 0):
            loss = self.calculate_total_loss(X, Y)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if len(losses) > 1 and losses[-1][1] > losses[-2][1]:
                learning_rate = learning_rate * 0.5
                print("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(Y)):
            self.sgd_step(X[i], Y[i], learning_rate)
            num_examples_seen += 1
    return losses 

Here: `sgd_step` does two things: 

1. Runs data through the network
2. Updates the weights

I'm going to copy the `rnn-from-scratch` repo [here](https://github.com/pangolulu/rnn-from-scratch), but modify it to work with LSTMs following [here](http://blog.varunajayasiri.com/numpy_lstm.html).

# Point

I want a structure that defines a series of operations and knows how to send values through the structure appropriately. 

What would that look like?


```python
# will have a number of nodes equal to the length of the input sequence
class LSTM_Node:
    ...
    def __init__(self, x_dim, h_dim)
    # store state as dictionary
    # initialize params object

    def forward(self, x_input, h_input, c_input):
        ...
        return x_out, h_out, c_out
    
    def forward(self, x_input, h_input, c_input):
        ...
        return x_out, h_out, c_out        
```

```python
# store all the params
class LSTM_Params:
    
    def __init__(self, x_dim, h_dim):
        # initialize params, diffs to correct shapes
        

    def update_params(self):

            
```

```python
class LSTM_Model:
    def __init__(self, sequence_length):
        # Initialize sequence of LSTM nodes
        # Learning rate etc.
        
    def forward_pass(self, x_batch):
        # Initialize x_in, h_in, c_in, run em through
        for node in nodes:
            x_in, h_in, c_in = node.forward(x_in, h_in, c_in)
            # Nodes are storing all the "h" and "c" information
        return x_out
    
    def loss(self, prediction, y_batch):
        return (prediction - y_batch) ** 2

    def loss_gradient(self, prediction, y_batch):
        y_batch = np.zeros_like(prediction)
        y_batch = 2 * (prediction - y_batch)
        return y_batch
                
    def backward_pass(self, loss_diff):
        # Initialize x_in, h_in, c_in, run em through
        for node in nodes:
            loss_diff, h_diff, c_diff = node.forward(loss_diff, h_diff, c_diff)
            # Nodes are storing all the "h" and "c" information       
        
        

```

Key part:


x_input = seq (len = 15)
y_output = seq (len = 15)



# Goal: 

Same "forward-loss-backward" API as before!!

In [2]:
import numpy as np
list(range(10, 5, -1))

[10, 9, 8, 7, 6]

In [3]:
def sigmoid(x, deriv=False):
    if deriv:
        return sigmoid(x) * (1 - sigmoid(x))
    else:
        return 1 / (1 + np.exp(-x))

def tanh(x, deriv=False):
    if deriv:
        return 1 - y * y
    else:
        return np.tanh(x)

In [5]:
class LSTM_Model:
    '''
    An LSTM model with one LSTM layer that feeds data through it and generates an output.
    '''
    def __init__(self, sequence_length, vocab_size, hidden_size, learning_rate):
        '''
        Initialize list of nodes of length the sequence length
        List the vocab size and the hidden size 
        Initialize the params
        '''
        self.nodes = [LSTM_Node(num_hidden, vocab_size) for x in range(sequence_length)]
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.start_H = np.zeros_like(hidden_size)
        self.start_C = np.zeros_like(hidden_size)
        self.params = LSTM_Params(vocab_size, hidden_size)
        self.learning_rate = learning_rate
        
    def forward(self, x_batch, first_iter=False):
        # Initialize x_in, h_in, c_in, run em through
        batch_size = x_batch.size[0]
        x_out = x_batch
        if first_iter:
            h_in = np.zeros_like(num_hidden)
            c_in = np.zeros_like(num_hidden)
        else:
            h_in = self.nodes[0].H
            c_in = self.nodes[0].C            
        for node in self.nodes:
            x_out, h_in, c_in = node.forward(x_out, h_in, c_in, self.params)
        return prediction

    def loss(self, prediction, y_batch):
        '''
        Return a gradient: how much our prediction influences how much we "missed" by.
        '''
        loss = (y_batch - prediction) ** 2
        y_grad = -1.0 * (y_batch - prediction)
        return y_grad
            
    def backward(self, loss_grad):
        '''
        Implements the "Backpropagation Through Time" algorithm.
        For each step in the sequence T moving backwards, backpropagate through some number
        K nodes
        
        '''
        # Initialize h_grad and c_grad
        # Initialize x_in, h_in, c_in, run em through
        H_grad = np.zeros_like(self.hidden_size)
        C_grad = np.zeros_like(self.hidden_size)
        T = self.sequence_length - 1
        K = 5 # BPTT length
        
        # BPTT
        num_iterations = T - K
        for n in range(num_iterations):
            for t in range(T-n, T-K-n, -1):
                Y_grad, H_grad, C_grad = \
                    self.nodes[t-n].backward(self, Y_grad, H_grad, C_grad, LSTM_params)
        
#         self.start_H = H_grad
#         self.start_C = C_grad
        return 


    def single_step(self, x, y, learning_rate):
        prediction = self.forward(x, first_iter=True)
        loss, loss_gradient = self.loss(x, y)
        self.backward(x, y)
        self.params.update_params(self.learning_rate)
        return

    
    def train(self, X, Y, learning_rate, epochs):
        num_examples_seen = 0
        losses = []
        for epoch in range(nepoch):
            if (epoch % evaluate_loss_after == 0):
                loss = self.loss(X, Y)
                losses.append((num_examples_seen, loss))
            for i in range(len(Y)):
                self.sgd_step(X[i], Y[i], learning_rate)
                num_examples_seen += 1
        return losses             

In [6]:
class LSTM_Node:
    '''
    An LSTM Node that takes in input and generates output. 
    Has a size of its hidden layers and a vocabulary size it expects.
    '''
    def __init__(self, num_hidden, vocab_size):
        self.state = LSTM_State()
        
    def forward(self, X_input, H_input, C_input, LSTM_params):

        self.Z = np.row_stack((X_input, H_input))

        # Step 2
        self.F_int = np.dot(self.Z, LSTM_params.W_f)
        self.F = sigmoid(self.F_int)

        # Step 3
        self.I_int = np.dot(self.Z, LSTM_params.W_i)
        self.I = sigmoid(self.I_int)

        # Step 4
        self.C_prop_int = np.dot(self.Z, LSTM_params.W_c)
        self.C_prop = tanh(self.C_prop_int)

        # Step 5 
        self.C_prop = tanh(self.C_prop_int)
        self.C_out = self.F * C_input + self.I * self.C_prop

        # Step 6
        self.O_int = np.dot(self.Z, LSTM_params.W_O)
        self.O = sigmoid(self.O_int)

        # Step 7
        self.C_act = tanh(self.C_out)
        self.H_out = self.O * self.C_act

        # Step 8
        self.X_out = np.dot(self.H, LSTM_params.W_v)

        return self.X_out, self.H_out, self.C_out


    def backward(self, Y_grad, H_grad, C_grad, LSTM_params):
        # Initialize the gradient for the words and the hidden layers:
        self.Z_diff = np.zeros_like(self.Z)
        
        # 2
        LSTM_params.W_V_diff += np.dot(Y_grad, self.H.T)

        # 3
        self.H_diff = np.dot(LSTM_params.W_V_diff.T, Y_grad)
        self.H_diff += H_grad

        # 4
        self.O_diff = self.H_diff * self.C_act

        # 4.5
        self.O_int_diff = sigmoid(self.O, deriv=True) * self.O
        
        # 5
        LSTM_params.W_O_diff += np.dot(self.O_int_diff, self.Z.T)
        self.Z_diff = np.dot(self.O_int_diff, LSTM_params.W_O.T)
        
        # 6
        self.C_diff = C_grad
        self.C_diff += self.H_diff * self.O * tanh(C_act, deriv=True)

        # 7
        self.C_prop_diff = self.C_diff * self.I

        # 7.5
        self.C_prop_int_diff = tanh(self.C_prop, deriv = True) * self.C_prop_diff

        # 8
        LSTM_params.W_C_diff += np.dot(self.C_prop_int_diff, self.Z.T)
        self.Z_diff += np.dot(self.C_prop_int, LSTM_params.W_C.T)
        
        # 9
        self.I_diff = self.C_diff * self.C_prop

        # 9.5
        self.I_int_diff = sigmoid(self.I, deriv=True) * self.I_diff

        # 10
        LSTM_params.W_I_diff += np.dot(self.I_int_diff, self.Z.T)
        self.Z_diff += np.dot(self.I_int, LSTM_params.W_I.T)
        
        # 11
        self.F_diff = self.C_diff * self.C

        # 11.5
        self.F_int_diff = sigmoid(self.F, deriv=True) * self.F_diff

        # 12
        LSTM_params.W_F_diff += np.dot(self.F_int, self.Z.T)        
        self.Z_diff += np.dot(self.F_int, LSTM_params.W_F.T)
        
        # 13
        C_grad = self.F * self.C_diff

        # 14
        X_grad = self.Z[:self.vocab_size, :]
        H_grad = self.Z[self.vocab_size:, :]        

        return H_grad, C_grad, X_grad

In [7]:
class LSTM_Params:
    
    def __init__(self, hidden_size, vocab_size):
        self.stack_size = hidden_size + vocab_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        self.W_F = np.random.normal(size=(self.stack_size, self.hidden_size))
        self.W_I = np.random.normal(size=(self.stack_size, self.hidden_size))
        self.W_C = np.random.normal(size=(self.stack_size, self.hidden_size))
        self.W_O = np.random.normal(size=(self.stack_size, self.hidden_size))
        self.W_V = np.random.normal(size=(self.hidden_size, self.vocab_size))
        
        self.W_F_diff = np.zeros_like(self.W_F)
        self.W_I_diff = np.zeros_like(self.W_I)
        self.W_C_diff = np.zeros_like(self.W_C)
        self.W_O_diff = np.zeros_like(self.W_O)
        self.W_V_diff = np.zeros_like(self.W_V)

    def update_params(self, learning_rate):
        self.W_F -= learning_rate * self.W_F_diff
        self.W_I -= learning_rate * self.W_I_diff
        self.W_C -= learning_rate * self.W_C_diff
        self.W_O -= learning_rate * self.W_O_diff
        self.W_V -= learning_rate * self.W_V_diff